# NN tension prediction for SUPERball
### &nbsp; &nbsp; &nbsp; Chiara Ercolani


Biases initialized to small value

In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import os
import collections
import matplotlib.pyplot as plt
import scipy.io

from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
#Combine data from all three datasets
dataset1=scipy.io.loadmat('2018-02-08_Biggester_slow_data.mat')
#dataset2=scipy.io.loadmat('2018-02-01_17_Just_motor22.mat')
#dataset3=scipy.io.loadmat('2018-01-30_16-07_Better_ReallyBig_Data.mat')


In [3]:
D=23   #number of features recorded at each time step(sequence length) [all motors except for motor 22]
effort=dataset1.get('effort')
#effort=np.concatenate((effort,dataset2.get('effort')))
#effort=np.concatenate((effort,dataset3.get('effort')))

#Exclude motor 22 from the features
effort1=effort[:,0:21]
#Also want to include the output (@24)
effort2=effort[:,22:25]
effortFeatures=np.concatenate((effort1,effort2),axis=1)
print(effortFeatures.shape)

(1127580, 24)


In [4]:
#Cross validation parameters
#Number of cross validation boxes
K=10
#Number of samples in each box
set_length=int(len(effort)/K)

In [5]:
#Various paramenters

#CHANGING BATCH SIZE
batch_size = 150
batch_len =(set_length*(K-1))//batch_size
margin=0.05                                     # accuracy margin
lr=0.00005                                      # learning rate (originally 0.0003)
epochNumber=100                                 # Number of epochs
desiredAccuracy=0.7
                                
H1= 15*D                                         # size of hidden state
radius=1/0.0155

print('train_data_len=',set_length*(K-1),' batch_size=',batch_size,' batch_len=',
      batch_len,' D=',D,'H1=',H1)

train_data_len= 1014822  batch_size= 150  batch_len= 6765  D= 23 H1= 345


In [6]:
#Input shape: (num_samples,seq_length,input_dimension)
#Output shape: (num_samples, target)
Xin= tf.placeholder(tf.float32,shape=[batch_size,D],name='Xin')
Ytarget = tf.placeholder(tf.float32,shape=[batch_size,1],name='Ytarget')

#Xavier initialization for weights 
#http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization
Wx1 = tf.get_variable("Wx1", shape=[D, H1],initializer=tf.contrib.layers.xavier_initializer()); print('Wx1=',Wx1.get_shape())
#Wx2 = tf.get_variable("Wx2", shape=[H1, H2],initializer=tf.contrib.layers.xavier_initializer()); print('Wx2=',Wx2.get_shape())
#Wx3 = tf.get_variable("Wx3", shape=[H2, H3],initializer=tf.contrib.layers.xavier_initializer()); print('Wx3=',Wx2.get_shape())
Wy = tf.get_variable("Wy", shape=[H1, 1],initializer=tf.contrib.layers.xavier_initializer()); print('Wy=',Wy.get_shape())

#Biases initalized to 0
bh1 = tf.Variable(tf.ones([1,H1])*0.001,name="bh1"); print('bh1=',bh1.get_shape())
#bh2 = tf.Variable(tf.zeros([1,H2]),name="bh2"); print('bh2=',bh2.get_shape())
#bh3 = tf.Variable(tf.zeros([1,H3]),name="bh3"); print('bh3=',bh3.get_shape())
by = tf.Variable(tf.ones([1,1])*0.001,name="by"); print('by=',by.get_shape())

Wx1= (23, 345)
Wy= (345, 1)
bh1= (1, 345)
by= (1, 1)


In [7]:
# NN implementation with ReLU function and one hidden layer

h_1=tf.nn.relu(tf.matmul(Xin,Wx1)+bh1)
#h_2=tf.nn.relu(tf.matmul(h_1,Wx2)+bh2)
#h_3=tf.nn.relu(tf.matmul(h_2,Wx3)+bh3)
y_=tf.add(tf.matmul(h_1,Wy),by,name='y_')

print('Ypredicted=',y_.get_shape())
print('Ytarget=',Ytarget.get_shape())

#Mean Absolute Error cost function

cost=tf.reduce_mean(tf.abs(tf.subtract(y_, Ytarget)))

#Optimizer used to implement backpropagation
#Adding decayed learning rate
#batch_var=tf.Variable(0)
#learning_rate=tf.train.exponential_decay(lr,batch_var*batch_size,len(train_features),0.95,staircase=True)
#optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

Ypredicted= (150, 1)
Ytarget= (150, 1)


In [ ]:
#ACCURACY INTERVAL
lower_bound =tf.greater_equal(y_,tf.subtract(Ytarget,margin))
upper_bound= tf.less_equal(y_,tf.add(Ytarget,margin))
correct=tf.equal(lower_bound,upper_bound)
#correct=tf.equal(cast_out_data,cast_out_pred)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
print('Accuracy measure set')

Accuracy measure set


In [ ]:
init_op = tf.global_variables_initializer()
rmse=[]
rmse_motor=[]
for k in range(0,K):
    print("CROSS VALIDATION ITERATION",k)
    # Initializing session    
    sess = tf.Session()
    sess.run(init_op)
    
    train_features=[]
    train_features2=[]
    test_features=[]
    train_target=[]
    test_target=[]
    
    
    #Define saver object to save NN
    locals()["saver_"+str(k)]=tf.train.Saver()
    train_features= effortFeatures[0:k*set_length,0:D]
    train_features=np.concatenate((train_features,effortFeatures[(k+1)*set_length:K*set_length,0:D]))
    
    test_features= effortFeatures[k*set_length:(k+1)*set_length,0:D]
                               

    train_target= -effortFeatures[0:k*set_length,D]
    train_target=np.concatenate((train_target,-effortFeatures[(k+1)*set_length:K*set_length,D]))
    
    test_target=-effortFeatures[k*set_length:(k+1)*set_length,D]
                           
                               
    #Reshaping data
    train_features=np.asarray(train_features)
    train_features=np.reshape(train_features,[len(train_features),D])
    test_features=np.asarray(test_features)
    test_features=np.reshape(test_features,[len(test_features),D])
    train_target=np.asarray(train_target)
    train_target=np.reshape(train_target,[len(train_target),1])
    test_target=np.asarray(test_target) 
    test_target=np.reshape(test_target,[len(test_target),1]) 
    
    #Regularize to radius
    train_target=train_target*radius
    test_target=test_target*radius

    costs = []
    accuracies=[]

    #setting up training
    #for epoch in range(epochNumber):
    acc=0
    epoch=0
    while(acc<desiredAccuracy or epoch<epochNumber):
        total_batch=int(len(train_features)/batch_size)
        for i in range(total_batch):   
            inp_data= train_features[i*batch_size:(i+1)*batch_size,:]
            out_data= train_target[i*batch_size:(i+1)*batch_size,:]
            _,acc,c = sess.run([optimizer,accuracy,cost],feed_dict={Xin: inp_data, Ytarget: out_data})
        print("Epoch: {}, Cost: {}, Accuracy: {}".format(epoch,c,acc))
        costs.append(c)
        accuracies.append(acc)
        epoch=epoch+1

    #Save session after training
    saver_path=locals()["saver_"+str(k)].save(sess,'results3/NN_tension_pred_saver',global_step=k)

    print ("\nTraining complete!")

    predicted=[]
    test_batch=int(len(test_features)/batch_size)

    #Test prediction with test dataset
    for j in range (test_batch):    
        inp_data = test_features[j*batch_size:(j+1)*batch_size,:]
        pred = sess.run(y_, feed_dict={Xin: inp_data})
        predicted=np.append(predicted,pred)

    #Plot prediction against real data from test dataset
    x=np.arange(0,len(test_target[0:(batch_size*test_batch)]))
    y_pred= predicted
    #invert because i invert the y_tar value when training
    y_tar= test_target[0:(batch_size*test_batch)]
    #y3=-test_features[:,21]*0.0175/0.008
    y_motor=effort[k*set_length:k*set_length+(batch_size*test_batch),21]/0.0155*2.5

    fig= plt.figure(figsize=(20,10))
    ax1=plt.subplot(211)
    ax1.plot(x,y_pred,'b',label='Predictions')
    ax1.plot(x,y_tar,'r',label='Targets')
    ax1.plot(x,y_motor,'g',label='Motor 22')
    ax1.legend(loc="upper right")

    plt.title('Prediction using 24 motor torques as features')

    plt.show()
    
    # Plot cost function and accuracy
    x=np.arange(0,epochNumber)
    y= costs

    fig= plt.figure(figsize=(10,10))
    ax1=plt.subplot(211)
    ax1.plot(x,y,'b',label='Cost')
    ax1.legend(loc="upper right")

    plt.title('Cost Fuction during Training')

    plt.show()

    # Plot cost function and accuracy
    x=np.arange(0,epochNumber)
    y= accuracies

    fig= plt.figure(figsize=(10,10))
    ax1=plt.subplot(211)
    ax1.plot(x,y,'r',label='Accuracy')
    ax1.legend(loc="upper left")

    plt.title('Accuracy during Training')

    plt.show()
    
    rmse=np.append(rmse,sqrt(mean_squared_error(y_pred,y_tar)))
    rmse_motor=np.append(rmse_motor,sqrt(mean_squared_error(y_motor,y_tar)))
    print("And now costs for iteration ", k)
    print(costs)
    print("And now accuracies for iteration ", k)
    print(accuracies)
    print("Root mean squared error of prediction",rmse[k])
    print("Root mean squared error of motor",rmse_motor[k])

    sess.close()
print(rmse)
print(rmse_motor)

CROSS VALIDATION ITERATION 0
Epoch: 0, Cost: 94.49454498291016, Accuracy: 0.0
Epoch: 1, Cost: 33.3946647644043, Accuracy: 0.0
Epoch: 2, Cost: 40.334754943847656, Accuracy: 0.0
Epoch: 3, Cost: 37.2031135559082, Accuracy: 0.0
Epoch: 4, Cost: 34.07879638671875, Accuracy: 0.0
Epoch: 5, Cost: 31.444053649902344, Accuracy: 0.0
Epoch: 6, Cost: 29.333499908447266, Accuracy: 0.0
Epoch: 7, Cost: 26.858158111572266, Accuracy: 0.0
Epoch: 8, Cost: 24.702356338500977, Accuracy: 0.0
Epoch: 9, Cost: 22.811443328857422, Accuracy: 0.0
Epoch: 10, Cost: 21.12898063659668, Accuracy: 0.0
Epoch: 11, Cost: 19.676929473876953, Accuracy: 0.0
Epoch: 12, Cost: 18.484638214111328, Accuracy: 0.0
Epoch: 13, Cost: 17.481679916381836, Accuracy: 0.0
Epoch: 14, Cost: 16.597583770751953, Accuracy: 0.0
Epoch: 15, Cost: 15.862868309020996, Accuracy: 0.0
Epoch: 16, Cost: 15.280301094055176, Accuracy: 0.0
Epoch: 17, Cost: 14.734557151794434, Accuracy: 0.0
Epoch: 18, Cost: 14.265751838684082, Accuracy: 0.0
Epoch: 19, Cost: 13

Epoch: 158, Cost: 2.2399368286132812, Accuracy: 0.013333333656191826
Epoch: 159, Cost: 2.205618143081665, Accuracy: 0.013333333656191826
Epoch: 160, Cost: 2.174504041671753, Accuracy: 0.013333333656191826
Epoch: 161, Cost: 2.128112316131592, Accuracy: 0.006666666828095913
Epoch: 162, Cost: 2.091526746749878, Accuracy: 0.0
Epoch: 163, Cost: 2.056020975112915, Accuracy: 0.0
Epoch: 164, Cost: 2.015434503555298, Accuracy: 0.006666666828095913
Epoch: 165, Cost: 1.975797176361084, Accuracy: 0.006666666828095913
Epoch: 166, Cost: 1.9357084035873413, Accuracy: 0.006666666828095913
Epoch: 167, Cost: 1.8958516120910645, Accuracy: 0.006666666828095913
Epoch: 168, Cost: 1.860059380531311, Accuracy: 0.013333333656191826
Epoch: 169, Cost: 1.8192285299301147, Accuracy: 0.046666666865348816
Epoch: 170, Cost: 1.7926108837127686, Accuracy: 0.03333333507180214
Epoch: 171, Cost: 1.7642525434494019, Accuracy: 0.013333333656191826
Epoch: 172, Cost: 1.7278153896331787, Accuracy: 0.006666666828095913
Epoch: 1

In [ ]:
# ADAM
#batch_size=800, lr=0.0001, K=10, H=4*D, epoch_number=100 
#[ 0.50253549  0.43781634  0.32040607  0.32329862  0.36412889  0.35812448
#  0.27207648  0.43003502  0.32162693  0.40849026]
#[ 0.70579643  0.64966416  0.50766536  0.50934368  0.59895063  0.61945065
#  0.48574498  0.67014488  0.48967202  0.61927773]
#
#batch_size=100, lr=0.0003, K=10, H=5*D, epoch_number=100
#[ 0.46218572  0.44220819  0.39805842  0.41903569  0.3219875   0.3759407
#  0.35630052  0.46268236  0.37257996  0.54600813]
#[ 0.70500703  0.64769705  0.50612557  0.50971187  0.59769723  0.61814086
#  0.48427049  0.66855334  0.4923125   0.61763899]


#batch_size=500, lr=0.0003, K=10, H=5*D, epoch_number=100
#[ 0.45112949  0.41166623  0.3488705   0.33144462  0.30662881  0.37951444
#  0.26394875  0.4951899   0.29257544  0.48440297]
#[ 0.70496692  0.64826301  0.50657057  0.50966813  0.59798115  0.61856779
#  0.48468212  0.66901232  0.49050164  0.61812131]

#batch_size=500, lr=0.0003, K=10, H=5*D, epoch_number=300
#[ 0.46730755  0.39162348  0.33621428  0.35487699  0.33413626  0.4338099
#  0.338667    0.46846256  0.3382114   0.55634947]
#[ 0.70496692  0.64826301  0.50657057  0.50966813  0.59798115  0.61856779
#  0.48468212  0.66901232  0.49050164  0.61812131]

##batch_size=200, lr=0.00005, K=10, H=15*D, epoch_number=100
#[ 0.43956565  0.39496273  0.30556959  0.32533406  0.33855176  0.37363014
#  0.28639368  0.46719669  0.31364863  0.42626489]
#[ 0.70492231  0.64797778  0.50634846  0.50969095  0.59783157  0.61834494
#  0.48447538  0.66878822  0.49126617  0.6178877 ]

#batch_size=200, lr=0.00005, K=10, H=20*D, epoch_number=100
#[ 0.45373054  0.39110336  0.32326501  0.33262925  0.35039483  0.40166448
#  0.25592091  0.4547832   0.30091716  0.44751162]
#[ 0.70492231  0.64797778  0.50634846  0.50969095  0.59783157  0.61834494
#  0.48447538  0.66878822  0.49126617  0.6178877 ]


#MINI BATCH GD
#batch_size=100, lr=0.00001, K=10, H=15*D, epoch_number=200
#[ 0.48721713,  0.43863498,  0.33641318,  0.33274214,  0.36547824,  0.35071493,
#  0.27092393,  0.42188746,  0.33233472,  0.42640026]
#[ 0.70500703  0.64769705  0.50612557  0.50971187  0.59769723  0.61814086
#  0.48427049  0.66855334  0.4923125   0.61763899]

#batch_size=50, lr=0.00001, K=10, H=15*D, epoch_number=100
#[ 0.48799102  0.45342051  0.33284949  0.32443756  0.35144045  0.34472716
#  0.28091073  0.42006393  0.33825867  0.42149133]
#[ 0.70509088  0.64755575  0.50601392  0.50971825  0.59763682  0.61804062
#  0.48416542  0.66842852  0.49275822  0.61751091]


#batch_size=50, lr=0.00001, K=10, H=18*D, epoch_number=100
#[ 0.49460297  0.45634525  0.32905642  0.32753956  0.35563032  0.35412292
#  0.27835449  0.42779808  0.33047787  0.41999861]
#[ 0.70509088  0.64755575  0.50601392  0.50971825  0.59763682  0.61804062
#  0.48416542  0.66842852  0.49275822  0.61751091]

#batch_size=50, lr=0.00001, K=10, H=10*D, epoch_number=100
#[ 0.48540827  0.44815957  0.33539943  0.33215633  0.37294727  0.34524683
#  0.26910294  0.42505782  0.34029048  0.42350589]
#[ 0.70509088  0.64755575  0.50601392  0.50971825  0.59763682  0.61804062
#  0.48416542  0.66842852  0.49275822  0.61751091]


#batch_size=50, lr=0.00001, K=7, H=15*D, epoch_number=100
[ 0.49522226  0.3746474   0.35643217  0.33235772  0.33975522  0.34873753
  0.41965114]
[ 0.69181523  0.56609945  0.54216891  0.58965845  0.54702666  0.5808668
  0.59657655]

#batch_size=50, lr=0.000005, K=10, H=15*D, epoch_number=200
[ 0.48740707  0.46531014  0.33256863  0.33262834  0.35801402  0.34543959
  0.28328477  0.41193634  0.33103235  0.42206953]
[ 0.70509088  0.64755575  0.50601392  0.50971825  0.59763682  0.61804062
  0.48416542  0.66842852  0.49275822  0.61751091]


GOOD ONE
#batch_size=50, lr=0.000005, K=10, H=15*D, epoch_number=300
# [0.47976632  0.45696714  0.33366897  0.3296227   0.35630887  0.33606939
#  0.28331008  0.41057156  0.32990863  0.42158835]
#[ 0.70509088  0.64755575  0.50601392  0.50971825  0.59763682  0.61804062
#  0.48416542  0.66842852  0.49275822  0.61751091]

In [12]:
ciao=[ 0.70509088,  0.64755575 , 0.50601392 , 0.50971825 , 0.59763682,  0.61804062,
  0.48416542 , 0.66842852 , 0.49275822 , 0.61751091]

In [13]:
print(np.mean(ciao))

0.584691931


In [35]:
effort=dataset1.get('effort')
#effort=np.concatenate((effort,dataset2.get('effort')))
#effort=np.concatenate((effort,dataset3.get('effort')))
print(effort.shape)
effort1=effort[:,0:21]
effort2=effort[:,21:D]
effort=np.concatenate((effort1,effort2),axis=1)
print(effort1.shape)
print(effort2.shape)
print(effort.shape)

(1127580, 25)
(1127580, 21)
(1127580, 2)
(1127580, 23)


think about SHIFTING UP cause the relu is only from 0 up!!!